# Projet : Réimplémentation et Amélioration de l'algorithme RLT (Reinforcement Learning Trees)

**Objectif :** Ce notebook structure le projet de réimplémentation en Python de l'algorithme RLT.

**Auteurs :** Kousay Najar, Hamza Farhani, Taoufik Krid, Wiem Ben M'Sahel, Rawen Mezzi, Mohamed Khayat

**Date :** 11/2025

## Phase 1 : Compréhension du Métier (Business Understanding)

Cette phase a pour but de définir les objectifs du point de vue du métier et de les traduire en un problème de data science bien défini.

### 1.1 Objectifs Métier (Business Objectives - BOs)
Les BOs décrivent la valeur ajoutée attendue d'un point de vue non technique.

- **BO1 : Augmenter la fiabilité des décisions grâce à une modélisation de nouvelle génération.** Développer un modèle qui surpasse les standards actuels sur les données complexes (fortes interactions, variables non pertinentes). L'objectif est de fournir des prédictions plus précises pour des processus critiques (évaluation du risque, ciblage marketing, maintenance prédictive), réduisant ainsi les erreurs coûteuses et augmentant la confiance dans les systèmes automatisés.

- **BO2 : Créer un avantage compétitif en développant un algorithme prédictif propriétaire.** Aller au-delà de la recherche existante pour construire un modèle unique, potentiellement plus performant ou plus rapide que les solutions standards. L'objectif est de doter l'entreprise d'un actif technologique différenciant, lui permettant d'optimiser ses opérations de manière plus efficace que ses concurrents et de s'attaquer à des problèmes jusqu'alors insolubles.

- **BO3 : Rendre les décisions de l'IA transparentes pour accélérer l'adoption et assurer la conformité.** Intégrer une couche d'explicabilité pour que chaque prédiction du modèle soit accompagnée d'une justification claire et compréhensible. Cela permet aux experts métier de faire confiance aux résultats, aux décideurs de gouverner l'usage de l'IA de manière responsable, et à l'entreprise de se conformer aux exigences réglementaires en matière de transparence algorithmique.

### 1.2 Objectifs Data Science (Data Science Objectives - DSOs)
Les DSOs sont les objectifs techniques qui, une fois atteints, permettront de réaliser les BOs.

- **DSO1 : Développer une implémentation Python validée de l'algorithme Reinforcement Learning Trees (RLT).** Traduire en code Python fonctionnel les mécanismes décrits dans la publication de Zhu et al. (2015). Le livrable sera une librairie interne contenant les classes `RLT_Tree` et `RLT_Forest`, implémentant fidèlement la sélection de variables par renforcement, le "muting", et les séparations par combinaisons linéaires. Le critère de succès sera la capacité du modèle à résoudre des problèmes de benchmark (ex: le problème du damier) de manière plus efficace qu'un Random Forest standard.

- **DSO2 : Augmenter la performance de l'algorithme RLT via l'expérimentation de modèles embarqués.** Mener une série d'expériences contrôlées pour remplacer le modèle embarqué standard de RLT (type Random Forest) par des alternatives plus performantes comme LightGBM ou XGBoost. L'objectif est de quantifier l'impact de ce changement sur la précision prédictive et la vitesse d'entraînement. Le critère de succès sera l'obtention d'une nouvelle version du modèle ("RLT-Boosted") démontrant une amélioration mesurable (ex: gain d'AUC, réduction du MSE ou du temps de calcul) par rapport à l'implémentation de base.

- **DSO3 : Intégrer une couche d'explicabilité (XAI) native au sein du modèle RLT.** Développer une méthode `.explain(X)` au sein de la classe `RLT_Forest` qui, pour une observation donnée, retourne la contribution de chaque variable à la prédiction finale. Le livrable technique sera cette méthode, qui produira une sortie structurée (ex: dictionnaire JSON) des attributions de features. Le critère de succès est la capacité de générer des explications locales qui sont à la fois stables (robustes à de petites variations d'input) et cohérentes avec l'importance globale des variables du modèle.

## Phase 2 : Compréhension des Données (Data Understanding)

Nous utiliserons des jeux de données synthétiques pour isoler et tester les fonctionnalités spécifiques, y compris la capacité d'explication (DSO3) et la performance sur structures complexes (DSO1/DSO2).

### 2.1 Jeu de données "Checkerboard"
**Objectif :** Valider DSO1. Ce jeu de données ne peut être résolu efficacement qu'avec des séparations diagonales, testant la fonctionnalité `combsplit`.

### 2.2 Jeu de données avec variables non-pertinentes et Vérité Terrain
**Objectif :** Valider DSO2 (robustesse) et DSO3 (explicabilité). Nous devons connaître exactement quelles variables sont importantes pour vérifier si la méthode `.explain()` les identifie correctement.

## Phase 3 : Préparation des Données (Data Preparation)

La préparation consistera principalement à diviser nos jeux de données synthétiques.

## Phase 4 : Modélisation (Modeling)

Cette phase est le cœur du projet. Nous implémentons RLT et ajoutons les extensions requises par DSO2 (modèles embarqués variés) et DSO3 (explicabilité).

## Phase 5 : Évaluation (Evaluation)

Nous validons ici les trois DSOs de manière distincte.

In [ ]:
# === Validation DSO1 : Reproduction RLT Standard ===
# Entraîner RLT (mode standard) sur 'Checkerboard' et comparer avec RandomForest sklearn.
# Critère : RLT doit avoir une accuracy significativement supérieure grâce au 'combsplit'.

# === Validation DSO2 : Benchmarking des Modèles Embarqués ===
# Comparer RLT_Standard (embedded='rf') vs RLT_Boosted (embedded='lightgbm').
# Mesurer : Temps d'entraînement et Précision/MSE sur les données bruitées.
# Critère : RLT_Boosted doit montrer un gain de performance ou de vitesse.

# === Validation DSO3 : Qualité de l'Explicabilité ===
# Utiliser .explain() sur le jeu de données bruité.
# Vérifier si la heatmap générée attribue une importance proche de 0 aux variables de bruit.
# Visualiser une 'Feature Heatmap' pour quelques prédictions.

## Phase 6 : Déploiement (Deployment)

Préparation du modèle pour l'intégration, en mettant l'accent sur l'explicabilité (BO3).

### 6.1 Sauvegarde du Modèle Optimisé
Sauvegarder la meilleure version du modèle (probablement celle issue de DSO2).

### 6.2 Interface Streamlit (avec XAI)
**Objectif :** Démonstrateur interactif incluant la transparence.

**Fonctionnalités :**
- Inputs utilisateur.
- Affichage Prédiction.
- **Feature Heatmap (BO3) :** Afficher graphiquement quelles variables ont poussé la décision (output de `.explain()`).

### 6.3 API FastAPI
**Objectif :** Intégration système.

**Endpoint :** `POST /predict`
- **Input :** Données JSON.
- **Output :** `{"prediction": value, "explanation": {feature_contributions}}`
- L'API retourne non seulement le résultat mais aussi le "pourquoi" (BO3).